In [2]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ------------------------------ --------- 61.4/81.1 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 81.1/81.1 kB 1.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=88623d5e96e7e63812e8ab72c1c3838bf338ffce97f3c908d25888b15d700902
  Stored in directory: c:\users\apurv\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [9]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://biddytarot.com/tarot-card-meanings/major-arcana/fool/")
docs= loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
documents

[Document(page_content='The Fool Tarot Card Meanings | Biddy Tarot                          Skip to main content Doors Open!Join the Biddy Tarot Certification Program Get Early Bird Pricing  Hit enter to search or ESC to closeClose Search         search  Menu     Free TrainingFree ReadingsLearnStart HereTarot Card MeaningsTarot 101Books & GuidesCoursesCommunityJoinLoginBlogPodcastShopGrow Your BizAboutMeet BrigitCareersReviewsContact & FAQDiversity PolicyLoginsearch Home > Major Arcana Tarot Card Meanings > The Fool Tarot Card MeaningsThe Fool Tarot Card Meanings Love this deck? Buy The Biddy Tarot Deck Now!The Fool KeywordsUPRIGHT: Beginnings, innocence, spontaneity, a free spiritREVERSED: Holding back, recklessness, risk-takingThe Fool DescriptionThe Fool is numbered 0 – the number of unlimited potential – and so does not have a specific place in the sequence of the Tarot cards. The Fool can be placed either at the beginning of the Major Arcana or at the end. The Major Arcana is ofte

In [12]:
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020B7E9F8A10>)

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"fool_search",
"Search about meaninng of the fool card. For any query about fool you must use this tool ")

In [14]:
retriever_tool.name

'fool_search'

In [15]:

## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [25]:
tools = [wiki,retriever_tool,arxiv]

### Agents

In [18]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [20]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### agents

In [21]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)


#### agent executer

In [22]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

from langhain

In [24]:
agent_executor.invoke({"input": "Tell me about the symbolisms in the fool"})



> Entering new AgentExecutor chain...

Invoking: `fool_search` with `{'query': 'symbolisms'}`


any device, anywhere life takes you150+ page Soul Meditations GuidebookPlus, Bonus Material:40-minute Fool’s Journey meditation + guidebook22 Creative Ways to Connect with the Major Arcana X  eBook: Tarot Card Meanings WorkbookCreate a deeply intuitive and personal connection to the Tarot cards with your very own Tarot Card Meanings Workbook. Here’s what you’ll find inside the workbook:Guidance for how to uncover personal meanings for each of the Tarot CardsWorksheets for each card, with space to record personal keywords, Tarot card symbolism, upright and reversed meanings, and interpretations for love, work, career, finance, well-being and spiritualityImages of the 78 Rider-Waite Tarot cards to deepen your understanding of their symbolismPlus, Bonus:Blank Tarot Card Keyword Chart to create your very own quick-reference guide X  eBook: The Ultimate Guide to Tarot Card MeaningsHere’s what y

{'input': 'Tell me about the symbolisms in the fool',
 'output': "The Fool card in Tarot symbolizes new beginnings, opportunity, and potential. The young man depicted on the card is at the outset of his journey, standing at the cliff's edge, ready to take his first step into the unknown. The Fool represents taking a leap of faith, trusting where the Universe is leading you, and following your heart, even if it seems like a crazy decision.\n\nThe loyal dog at The Fool's feet symbolizes protection and encouragement, urging him to move forward and learn the lessons he needs to learn. The mountains in the background represent the challenges that lie ahead, but The Fool is focused on the present moment and starting his expedition without worrying about what's to come.\n\nWhen The Fool card appears reversed, it suggests that you may have a new project or opportunity in mind but are not ready to bring it to fruition yet. You might feel unprepared, lacking the necessary skills or resources, or

In [27]:
response = agent_executor.invoke({"input": "Tell me about the symbolisms in the fool"})



> Entering new AgentExecutor chain...

Invoking: `fool_search` with `{'query': 'symbolisms'}`


any device, anywhere life takes you150+ page Soul Meditations GuidebookPlus, Bonus Material:40-minute Fool’s Journey meditation + guidebook22 Creative Ways to Connect with the Major Arcana X  eBook: Tarot Card Meanings WorkbookCreate a deeply intuitive and personal connection to the Tarot cards with your very own Tarot Card Meanings Workbook. Here’s what you’ll find inside the workbook:Guidance for how to uncover personal meanings for each of the Tarot CardsWorksheets for each card, with space to record personal keywords, Tarot card symbolism, upright and reversed meanings, and interpretations for love, work, career, finance, well-being and spiritualityImages of the 78 Rider-Waite Tarot cards to deepen your understanding of their symbolismPlus, Bonus:Blank Tarot Card Keyword Chart to create your very own quick-reference guide X  eBook: The Ultimate Guide to Tarot Card MeaningsHere’s what y

In [28]:
response['output']

"The Fool card in Tarot symbolizes new beginnings, opportunity, and potential. The young man depicted on the card is at the outset of his journey, standing at the cliff's edge, ready to take his first step into the unknown. The dog at his heels represents loyalty and protection, encouraging him to move forward and learn the lessons he needs to learn. The mountains in the background symbolize the challenges that lie ahead, but The Fool is focused on starting his expedition without worrying about them.\n\nIn the upright position, The Fool signifies a time of new beginnings where you are called to commit yourself and follow your heart, even if the leap of faith seems daunting. It's a card of trust in the Universe and taking a chance on the unknown.\n\nIn the reversed position, The Fool suggests that you may have a new project or opportunity but are hesitant to move forward. You might feel unprepared, lack confidence, or fear the unknown outcome. This hesitation can lead to a standstill, w